In [1]:
import pandas as pd

transactions_t_data = pd.read_csv('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/transactions_train.csv')
display(transactions_t_data.head())

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,19765990,2017-07-07 09:53:00,1581,2519,$12.35,Chip Transaction,20519,Arnold,MO,63010.0,5942,NaN
1,22160255,2018-11-26 17:44:00,1862,4049,$58.29,Chip Transaction,98648,Des Moines,IA,50313.0,5814,NaN
2,17566794,2016-03-26 12:42:00,1967,3367,$11.03,Chip Transaction,46978,Lake Forest,CA,92630.0,5411,NaN
3,17318690,2016-02-01 08:30:00,921,3457,$85.74,Chip Transaction,63701,Rush,NY,14543.0,5411,NaN
4,20994060,2018-03-24 14:42:00,456,2800,$13.43,Chip Transaction,83271,Estero,FL,33928.0,4214,NaN


In [3]:
train_fraude_labels_data = pd.read_json('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/train_fraud_labels.json')
display(train_fraude_labels_data.head())

,target
17172593,No
17172612,No
17172621,No
17172635,No
17172637,No


In [5]:
cards_data = pd.read_csv('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/cards_data.csv')
display(cards_data.head())

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [6]:
users_data = pd.read_csv('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/users_data.csv')
display(users_data.head())

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [10]:
import json

with open('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/mcc_codes.json') as f:
    data = json.load(f)

mcc_codes_data = pd.DataFrame(list(data.items()), columns=['MCC_Code', 'Description'])
display(mcc_codes_data.head())

,MCC_Code,Description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [11]:
evaluation_features_data = pd.read_csv('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data/evaluation_features.csv')
display(evaluation_features_data.head())

,transaction_id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,23042130,2019-06-01 14:41:00,900,3412,$118.61,Swipe Transaction,61195,Panama City,FL,32401.0,5541,NaN
1,22702314,2019-03-21 13:45:00,1741,3025,$-204.00,Chip Transaction,49637,Tucson,AZ,85736.0,3509,NaN
2,22838524,2019-04-19 11:53:00,1081,3892,$-99.00,Chip Transaction,43293,Aurora,CO,80013.0,5499,NaN
3,23477219,2019-09-01 11:46:00,1740,2382,$28.44,Chip Transaction,26032,Mukwonago,WI,53149.0,7210,NaN
4,23066897,2019-06-06 21:24:00,1207,6056,$32.02,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN


In [8]:
from dataclasses import dataclass
from pathlib import Path
import os, json, pandas as pd, numpy as np
pd.set_option("display.max_columns", 120)

In [9]:
@dataclass(frozen=True)
class Paths:
    DATA_DIR: Path = Path('/Users/rayanhamadeh/Library/CloudStorage/OneDrive-DeVinci/ESILV/S9/BI Pipeline/hackthon data')

@dataclass
class Schema:
    TX_FILE: str = 'transactions_train.csv'
    ID_COL: str = 'transaction_id'
    TIME_COL: str = 'date'
    USER_COL: str = 'client_id'
    CARD_COL: str = 'card_id'
    AMOUNT_COL: str = 'amount'

    MERCHANT_ID: str = 'merchant_id'
    MERCHANT_CITY: str = 'merchant_city'
    MERCHANT_STATE: str = 'merchant_state'
    ZIP_COL: str = 'zip'
    MCC_COL: str = 'mcc'
    USE_CHIP: str = 'use_chip'
    ERRORS_COL: str = 'errors'

P = Paths()
S = Schema()


In [11]:
REQ_COLS = {
    S.ID_COL, S.TIME_COL, S.USER_COL, S.CARD_COL, S.AMOUNT_COL,
    S.MERCHANT_ID, S.MERCHANT_CITY, S.MERCHANT_STATE, S.ZIP_COL, S.MCC_COL, S.USE_CHIP, S.ERRORS_COL
}

def _clean_amount(series: pd.Series) -> pd.Series:
    return (
        series.astype(str)
        .str.strip()
        .str.replace(r"[^0-9]", "", regex=True)
        .replace({"": np.nan, ".": np.nan})
        .astype(float)
    )

def _clean_zip(series: pd.Series) -> pd.Series:
    # we need to keep zip as zero-padded (5+ digits) so that we don't lose leading zeros
    s = series.astype(str).str.split(".").str[0]  # remove decimal part if any
    s = s.str.replace(r"[^0-9]", "", regex=True)  # keep only digits
    return s.str.zfill(5)

def load_transactions(path_dir: Path, schema: Schema) -> pd.DataFrame:
    use_dtypes = {
        schema.ID_COL: "int64",
        schema.USER_COL: "int64",
        schema.CARD_COL: "int64",
        schema.MERCHANT_ID: "int64",
        schema.MERCHANT_CITY: "string",
        schema.MERCHANT_STATE: "string",
        schema.ZIP_COL: "string",
        schema.MCC_COL: "int64",
        schema.USE_CHIP: "string",
        schema.ERRORS_COL: "string"
    }

    df = pd.read_csv(path_dir / schema.TX_FILE, dtype=use_dtypes, keep_default_na=True)

    # Validate schema
    missing = REQ_COLS - set(df.columns)
    assert not missing, f"Missing required columns: {missing}"

    df[schema.TIME_COL] = pd.to_datetime(df[schema.TIME_COL], errors="coerce")
    assert df[schema.TIME_COL].notna().any(), "All parsed dates are NaT — check date format"

    #amount cleenup
    df[schema.AMOUNT_COL] = _clean_amount(df[schema.AMOUNT_COL])

    #zip cleanup
    df[schema.ZIP_COL] = _clean_zip(df[schema.ZIP_COL])

    #normalize categorical columns
    for c in [schema.MERCHANT_CITY, schema.MERCHANT_STATE, schema.USE_CHIP, schema.ERRORS_COL]:
        df[c] = df[c].str.lower().str.strip()

    # category compression for high cardinality text (to be encoded later)
    for c in [schema.MERCHANT_CITY, schema.MERCHANT_STATE, schema.USE_CHIP]:
        df[c] = df[c].astype("category")
    
    # id columns to string (to avoid scientifix notation)
    for c in [schema.ID_COL, schema.USER_COL, schema.CARD_COL, schema.MERCHANT_ID]:
        df[c] = df[c].astype("string")

    #MCC as category
    df[schema.MCC_COL] = df[schema.MCC_COL].astype("int64"). astype("category")

    # some quality checks
    print(f"Transactions: {len(df):,} rows")
    print(df[[schema.ID_COL, schema.TIME_COL, schema.USER_COL, schema.CARD_COL]].head(3))
    print("\nNulls (%):")
    print((df.isna().mean() * 100).round(3).sort_values(ascending=False).to_frame("null_pct").T)
    return df

transactions = load_transactions(P.DATA_DIR, S)

Transactions: 210,000 rows
  transaction_id                date client_id card_id
0       19765990 2017-07-07 09:53:00      1581    2519
1       22160255 2018-11-26 17:44:00      1862    4049
2       17566794 2016-03-26 12:42:00      1967    3367

Nulls (%):
          errors  merchant_state  transaction_id  date  client_id  card_id  \
null_pct  98.441           12.11             0.0   0.0        0.0      0.0   

          amount  use_chip  merchant_id  merchant_city  zip  mcc  
null_pct     0.0       0.0          0.0            0.0  0.0  0.0  
